In [211]:
### Content

# 1- Libraries
# 2- API Integration
# 3- Download the Files
# 4- Read the Files
# 5.Save Original Data

# 6- Exploratory Data Analysis
# 6.1- Missing Data
# 6.2- Statistical Analysis
# 6.2- Histograms
# 6.3- Distributions
# 6.4- Outliers

# 7- Data Cleaning
# 7.1- Handling with Date Columns
# 7.2- Drop Unnecessary Columns
# 7.3- Encoding
# 7.4- Handling Missing Data

# 7.5 Drop Outliers 1
# 7.6 Check Outliers Test 1

# 7.7 Drop Outliers 2
# 7.8 Check Outliers Test 2

# 7.9 Drop Outliers 3
# 7.10 Check Outliers Test 3

# 7.11 Drop Outliers 4
# 7.12 Check Outliers Test 4    

# 7.13 Data Normalization
# 7.14 Check Distributions 2
# 7.15 Correlation Test

# 8.1- Drop Outliers
# 8.1.1- Balance
# 8.1.2- EstimatedSalary
# 8.2- Standart Scaler
# 5.3 Drop Unnecessary Columns 





# 7- Building Model-0 // Benchmark
# 7.1- Logistic Regression




# 9- Building Alternative Models
# 9.1- Model 1 - Decision Trees
# 9.2- Model 2 - Random Forest
# 9.3- Model 3 - Gradient Boosting Machines
# 9.4- Model 4 - xgboost

# 10- Handling Imbalanced Data
# 11- Re-Train Best Performed Model with Balanced Dataset 

# 12- Feature Engineering
# 12.1 Correlation Test
# 12.2 Recursive Feature Elimination

# 13- Hyperparameter Tuning
# 13.1- Grid Search
# 13.2- Random Search

# 14- Cross Validation
# 15- Prediction

### 1- Libraries

In [212]:
### 1- Libraries

# Data Manipulation and Cleaning Libraries
import pandas as pd  # For data manipulation and data frames
import numpy as np  # For numerical operations and arrays

# File and System Operations Libraries
import os  # For operating system interactions, like file handling
import sys
import zipfile  # For working with zip files

# Dataset Access Libraries
from kaggle.api.kaggle_api_extended import KaggleApi  # For accessing datasets from Kaggle
from config import *  # Importing custom configurations
from model_eval_func import *  # Custom model evaluation functions

# Data Visualization Libraries
import seaborn as sns  # For advanced data visualization
from matplotlib import pyplot as plt  # For plotting graphs and charts
import matplotlib  # For customizing matplotlib settings


import math
from scipy import stats

# Machine Learning Libraries
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Visible All Rows and Columns 
pd.set_option("display.max_rows", None, "display.max_columns", None)


### 2- API

In [213]:
# Specify the correct file paths
raw_data_directory = path_1
extracted_path     = path_2

In [214]:
# Kaggle API
api = KaggleApi()
api.authenticate()

competition = comp
api.competition_download_files(competition, path=raw_data_directory)


### 3- Download the Files

In [215]:
# Download the file
zip_file_path = os.path.join(raw_data_directory, 'sberbank-russian-housing-market.zip')  # Burada dosya adını doğru şekilde belirtin

# Extract the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(raw_data_directory)

print(f"Zip Files Saved...")

Zip Files Saved...


### 4- Read the Files

In [216]:
# Klasörün içindeki dosyaları listeleme
if os.path.exists(raw_data_directory) and os.path.isdir(raw_data_directory):
    files = os.listdir(raw_data_directory)
    print("Files in the Raw Folder:")
    print(" ")
    for file in files:
        print(file)
else:
    print("Belirtilen klasör yok veya bir dizin değil.")

Files in the Raw Folder:
 
data_dictionary.txt
extracted
macro.csv.zip
sample_submission.csv.zip
sberbank-russian-housing-market.zip
test.csv.zip
train.csv.zip


In [217]:
# Create a directory for the extracted files
os.makedirs(extracted_path, exist_ok=True)

# List files in the directory
files = os.listdir(path_1)

# Extract zip files
for file in files:
    if file.endswith('.zip'):
        file_path = os.path.join(path_1, file)
        
        # Open the zip file
        with zipfile.ZipFile(file_path, 'r') as zip_ref:
            zip_ref.extractall(extracted_path)

print("Extraction complete.")


Extraction complete.


In [218]:
# List files in the extracted directory
files = os.listdir(extracted_path)

print("Dataframes: ")
print("")

# Read only the test and train CSV files and save them as variables
for file in files:
    if file.endswith('.csv') and ('test' in file or 'train' in file):
        file_path = os.path.join(extracted_path, file)
        print(f"Reading file: {file}")
        
        # Check if file exists
        if not os.path.exists(file_path):
            print(f"File does not exist: {file}")
            continue
        
        # Read the CSV file
        try:
            df = pd.read_csv(file_path, encoding='utf-8')
        except UnicodeDecodeError:
            try:
                df = pd.read_csv(file_path, encoding='latin1')
            except pd.errors.EmptyDataError:
                print(f"EmptyDataError: No columns to parse from file {file}")
                continue
            except Exception as e:
                print(f"Unexpected error while reading {file}: {e}")
                continue
        except pd.errors.EmptyDataError:
            print(f"EmptyDataError: No columns to parse from file {file}")
            continue
        except Exception as e:
            print(f"Unexpected error while reading {file}: {e}")
            continue

        # Determine variable name using the file name
        var_name = os.path.splitext(file)[0] + '_df'
        
        # Save the DataFrame using globals() function
        globals()[var_name] = df
        
        print(f"{var_name} saved...")

Dataframes: 

Reading file: test.csv
test_df saved...
Reading file: train.csv
train_df saved...


### 5- Save Original Data

In [219]:
# Combine dataframes
df = pd.concat([train_df, test_df])

In [220]:
# Copy the original dataframes
df_2 = df.copy()

### 6- Exploratory Data Analysis

In [221]:
# Drop id column
df_2 = df_2.drop("id", axis=1)

In [222]:
# Check datatype and blanks
df_2.info(verbose=True, show_counts=True)
df_2.head(2)

<class 'pandas.core.frame.DataFrame'>
Index: 38133 entries, 0 to 7661
Data columns (total 291 columns):
 #    Column                                 Non-Null Count  Dtype  
---   ------                                 --------------  -----  
 0    timestamp                              38133 non-null  object 
 1    full_sq                                38133 non-null  float64
 2    life_sq                                30574 non-null  float64
 3    floor                                  37966 non-null  float64
 4    max_floor                              28561 non-null  float64
 5    material                               28561 non-null  float64
 6    build_year                             23479 non-null  float64
 7    num_room                               28561 non-null  float64
 8    kitch_sq                               28561 non-null  float64
 9    state                                  23880 non-null  float64
 10   product_type                           38100 non-null  object 

,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,product_type,sub_area,area_m,raion_popul,green_zone_part,indust_part,children_preschool,preschool_quota,preschool_education_centers_raion,children_school,school_quota,school_education_centers_raion,school_education_centers_top_20_raion,hospital_beds_raion,healthcare_centers_raion,university_top_20_raion,sport_objects_raion,additional_education_raion,culture_objects_top_25,culture_objects_top_25_raion,shopping_centers_raion,office_raion,thermal_power_plant_raion,incineration_raion,oil_chemistry_raion,radiation_raion,railroad_terminal_raion,big_market_raion,nuclear_reactor_raion,detention_facility_raion,full_all,male_f,female_f,young_all,young_male,young_female,work_all,work_male,work_female,ekder_all,ekder_male,ekder_female,0_6_all,0_6_male,0_6_female,7_14_all,7_14_male,7_14_female,0_17_all,0_17_male,0_17_female,16_29_all,16_29_male,16_29_female,0_13_all,0_13_male,0_13_female,raion_build_count_with_material_info,build_count_block,build_count_wood,build_count_frame,build_count_brick,build_count_monolith,build_count_panel,build_count_foam,build_count_slag,build_count_mix,raion_build_count_with_builddate_info,build_count_before_1920,build_count_1921-1945,build_count_1946-1970,build_count_1971-1995,build_count_after_1995,ID_metro,metro_min_avto,metro_km_avto,metro_min_walk,metro_km_walk,kindergarten_km,school_km,park_km,green_zone_km,industrial_km,water_treatment_km,cemetery_km,incineration_km,railroad_station_walk_km,railroad_station_walk_min,ID_railroad_station_walk,railroad_station_avto_km,railroad_station_avto_min,ID_railroad_station_avto,public_transport_station_km,public_transport_station_min_walk,water_km,water_1line,mkad_km,ttk_km,sadovoe_km,bulvar_ring_km,kremlin_km,big_road1_km,ID_big_road1,big_road1_1line,big_road2_km,ID_big_road2,railroad_km,railroad_1line,zd_vokzaly_avto_km,ID_railroad_terminal,bus_terminal_avto_km,ID_bus_terminal,oil_chemistry_km,nuclear_reactor_km,radiation_km,power_transmission_line_km,thermal_power_plant_km,ts_km,big_market_km,market_shop_km,fitness_km,swim_pool_km,ice_rink_km,stadium_km,basketball_km,hospice_morgue_km,detention_facility_km,public_healthcare_km,university_km,workplaces_km,shopping_centers_km,office_km,additional_education_km,preschool_km,big_church_km,church_synagogue_km,mosque_km,theater_km,museum_km,exhibition_km,catering_km,ecology,green_part_500,prom_part_500,office_count_500,office_sqm_500,trc_count_500,trc_sqm_500,cafe_count_500,cafe_sum_500_min_price_avg,cafe_sum_500_max_price_avg,cafe_avg_price_500,cafe_count_500_na_price,cafe_count_500_price_500,cafe_count_500_price_1000,cafe_count_500_price_1500,cafe_count_500_price_2500,cafe_count_500_price_4000,cafe_count_500_price_high,big_church_count_500,church_count_500,mosque_count_500,leisure_count_500,sport_count_500,market_count_500,green_part_1000,prom_part_1000,office_count_1000,office_sqm_1000,trc_count_1000,trc_sqm_1000,cafe_count_1000,cafe_sum_1000_min_price_avg,cafe_sum_1000_max_price_avg,cafe_avg_price_1000,cafe_count_1000_na_price,cafe_count_1000_price_500,cafe_count_1000_price_1000,cafe_count_1000_price_1500,cafe_count_1000_price_2500,cafe_count_1000_price_4000,cafe_count_1000_price_high,big_church_count_1000,church_count_1000,mosque_count_1000,leisure_count_1000,sport_count_1000,market_count_1000,green_part_1500,prom_part_1500,office_count_1500,office_sqm_1500,trc_count_1500,trc_sqm_1500,cafe_count_1500,cafe_sum_1500_min_price_avg,cafe_sum_1500_max_price_avg,cafe_avg_price_1500,cafe_count_1500_na_price,cafe_count_1500_price_500,cafe_count_1500_price_1000,cafe_count_1500_price_1500,cafe_count_1500_price_2500,cafe_count_1500_price_4000,cafe_count_1500_price_high,big_church_count_1500,church_count_1500,mosque_count_1500,leisure_count_1500,sport_count_1500,market_count_1500,green_part_2000,prom_part_2000,office_count_2000,office_sqm_2000,trc_count_2000,trc_sqm_2000,cafe_count_2000,cafe_sum_2000_min_price_avg,cafe_sum_2000_max_price_avg,c

#### 6.1- Missing Data

In [223]:
# Count missing data
missing_data = df_2.isnull().sum()

# Missing data columns
missing_data = missing_data[missing_data > 0]

missing_data

life_sq                                   7559
floor                                      167
max_floor                                 9572
material                                  9572
build_year                               14654
num_room                                  9572
kitch_sq                                  9572
state                                    14253
product_type                                33
preschool_quota                           8284
school_quota                              8280
hospital_beds_raion                      17859
raion_build_count_with_material_info      6209
build_count_block                         6209
build_count_wood                          6209
build_count_frame                         6209
build_count_brick                         6209
build_count_monolith                      6209
build_count_panel                         6209
build_count_foam                          6209
build_count_slag                          6209
build_count_m

In [224]:
# Calculate the number of missing values
missing_data = df_2.isnull().sum()

# Calculate the percentage of missing values
missing_data_percentage = (df_2.isnull().sum() / len(df_2)) * 100

# Create a DataFrame that shows both the number and the percentage of missing values
missing_data_df = pd.DataFrame({'Missing Values': missing_data, 'Percentage': missing_data_percentage})

# Filter to show only columns where the number of missing values is greater than 0
missing_data_df = missing_data_df[missing_data_df['Missing Values'] > 0]

# Sort the DataFrame by the number of missing values in descending order
missing_data_df = missing_data_df.sort_values(by='Missing Values', ascending=False)

missing_data_df


,Missing Values,Percentage
hospital_beds_raion,17859,46.833451
cafe_sum_500_min_price_avg,16440,43.112265
cafe_avg_price_500,16440,43.112265
cafe_sum_500_max_price_avg,16440,43.112265
build_year,14654,38.428658
state,14253,37.377075
max_floor,9572,25.101618
material,9572,25.101618
num_room,9572,25.101618
kitch_sq,9572,25.101618


The dataset contains a significant amount of missing data, particularly in columns such as hospital_beds_raion (46.83%) and various cafe-related pricing metrics (around 43%). Additionally, important columns like build_year and state have over 30% missing values. This high percentage of missing data could impact the reliability and accuracy of any analysis, and it may require careful handling through imputation or exclusion strategies.

#### 6.2- Statistical Analysis

In [225]:
# Statistical Distribution
df_2.describe()

,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,area_m,raion_popul,green_zone_part,indust_part,children_preschool,preschool_quota,preschool_education_centers_raion,children_school,school_quota,school_education_centers_raion,school_education_centers_top_20_raion,hospital_beds_raion,healthcare_centers_raion,university_top_20_raion,sport_objects_raion,additional_education_raion,culture_objects_top_25_raion,shopping_centers_raion,office_raion,full_all,male_f,female_f,young_all,young_male,young_female,work_all,work_male,work_female,ekder_all,ekder_male,ekder_female,0_6_all,0_6_male,0_6_female,7_14_all,7_14_male,7_14_female,0_17_all,0_17_male,0_17_female,16_29_all,16_29_male,16_29_female,0_13_all,0_13_male,0_13_female,raion_build_count_with_material_info,build_count_block,build_count_wood,build_count_frame,build_count_brick,build_count_monolith,build_count_panel,build_count_foam,build_count_slag,build_count_mix,raion_build_count_with_builddate_info,build_count_before_1920,build_count_1921-1945,build_count_1946-1970,build_count_1971-1995,build_count_after_1995,ID_metro,metro_min_avto,metro_km_avto,metro_min_walk,metro_km_walk,kindergarten_km,school_km,park_km,green_zone_km,industrial_km,water_treatment_km,cemetery_km,incineration_km,railroad_station_walk_km,railroad_station_walk_min,ID_railroad_station_walk,railroad_station_avto_km,railroad_station_avto_min,ID_railroad_station_avto,public_transport_station_km,public_transport_station_min_walk,water_km,mkad_km,ttk_km,sadovoe_km,bulvar_ring_km,kremlin_km,big_road1_km,ID_big_road1,big_road2_km,ID_big_road2,railroad_km,zd_vokzaly_avto_km,ID_railroad_terminal,bus_terminal_avto_km,ID_bus_terminal,oil_chemistry_km,nuclear_reactor_km,radiation_km,power_transmission_line_km,thermal_power_plant_km,ts_km,big_market_km,market_shop_km,fitness_km,swim_pool_km,ice_rink_km,stadium_km,basketball_km,hospice_morgue_km,detention_facility_km,public_healthcare_km,university_km,workplaces_km,shopping_centers_km,office_km,additional_education_km,preschool_km,big_church_km,church_synagogue_km,mosque_km,theater_km,museum_km,exhibition_km,catering_km,green_part_500,prom_part_500,office_count_500,office_sqm_500,trc_count_500,trc_sqm_500,cafe_count_500,cafe_sum_500_min_price_avg,cafe_sum_500_max_price_avg,cafe_avg_price_500,cafe_count_500_na_price,cafe_count_500_price_500,cafe_count_500_price_1000,cafe_count_500_price_1500,cafe_count_500_price_2500,cafe_count_500_price_4000,cafe_count_500_price_high,big_church_count_500,church_count_500,mosque_count_500,leisure_count_500,sport_count_500,market_count_500,green_part_1000,prom_part_1000,office_count_1000,office_sqm_1000,trc_count_1000,trc_sqm_1000,cafe_count_1000,cafe_sum_1000_min_price_avg,cafe_sum_1000_max_price_avg,cafe_avg_price_1000,cafe_count_1000_na_price,cafe_count_1000_price_500,cafe_count_1000_price_1000,cafe_count_1000_price_1500,cafe_count_1000_price_2500,cafe_count_1000_price_4000,cafe_count_1000_price_high,big_church_count_1000,church_count_1000,mosque_count_1000,leisure_count_1000,sport_count_1000,market_count_1000,green_part_1500,prom_part_1500,office_count_1500,office_sqm_1500,trc_count_1500,trc_sqm_1500,cafe_count_1500,cafe_sum_1500_min_price_avg,cafe_sum_1500_max_price_avg,cafe_avg_price_1500,cafe_count_1500_na_price,cafe_count_1500_price_500,cafe_count_1500_price_1000,cafe_count_1500_price_1500,cafe_count_1500_price_2500,cafe_count_1500_price_4000,cafe_count_1500_price_high,big_church_count_1500,church_count_1500,mosque_count_1500,leisure_count_1500,sport_count_1500,market_count_1500,green_part_2000,prom_part_2000,office_count_2000,office_sqm_2000,trc_count_2000,trc_sqm_2000,cafe_count_2000,cafe_sum_2000_min_price_avg,cafe_sum_2000_max_price_avg,cafe_avg_price_2000,cafe_count_2000_na_price,cafe_count_2000_price_500,cafe_count_2000_price_1000,cafe_count_2000_price_1500,cafe_count_2000_price_2500,cafe_count_2000_price_4000,cafe_count_2000_price_high,big_church_count_2000,church_count_2000,mosque_count_2000,leisure_coun

### Observations:

Standard Deviation Issues: 
The data has high standard deviations, indicating that the data is not normally distributed.

Unit Differences: 
Different columns are measured in different units, leading to inconsistencies.

Scaling and Transformation Needed: 
The data needs to be scaled and transformed to be suitable for modeling.

#### 6.3- Histograms

In [226]:
# List all the numeric columns
numeric_columns = df_2.select_dtypes(include=['float64', 'int64']).columns

In [227]:
'''
# Define a function to plot histograms in groups
def plot_histograms(df_2, cols, n_cols=3, figsize=(20, 15)):
    n_rows = (len(cols) + n_cols - 1) // n_cols
    fig, axes = plt.subplots(n_rows, n_cols, figsize=figsize)
    axes = axes.flatten()
    
    for i, col in enumerate(cols):
        df[col].hist(bins=30, ax=axes[i])
        axes[i].set_title(col)
    
    for j in range(i + 1, len(axes)):
        fig.delaxes(axes[j])
    
    plt.tight_layout()
    plt.show()

# Plot histograms in groups of 10 columns
for i in range(0, len(numeric_columns), 10):
    plot_histograms(df_2, numeric_columns[i:i+10])
'''
print('')


#### 6.3- Check Distributions 1

In [228]:
'''
# Check the visualization
df_sampled = df_2.sample(frac=0.1)  # Verinin %10'unu alıyoruz
num_plots = len(numeric_columns)
plots_per_page = 16  # 4x4 yerleşim
num_pages = math.ceil(num_plots / plots_per_page)

for page in range(num_pages):
    plt.figure(figsize=(15, 15))
    
    start_idx = page * plots_per_page
    end_idx = min(start_idx + plots_per_page, num_plots)
    
    for i, column in enumerate(numeric_columns[start_idx:end_idx], 1):
        plt.subplot(4, 4, i)
        sns.histplot(df_sampled[column], kde=False)
        plt.title(column)
    
    plt.tight_layout()
    plt.show()
'''
print('')


The histograms above indicate that the distributions of the variables are heavily skewed and not normally distributed.

#### 6.4- Outliers Checking

In [229]:
'''
# Sütunları küçük gruplar halinde bölmek için adım boyutunu belirleyin
step = 10  # Aynı anda kaç sütun gösterileceğini belirler
num_columns = len(df_2.columns)

for i in range(0, num_columns, step):
    plt.figure(figsize=(12, 6))  # Grafik boyutunu ayarlayın
    sns.boxplot(data=df_2.iloc[:, i:i+step])
    plt.xticks(rotation=45, fontsize=8)
    plt.show()
'''
print('')

### 7- Data Cleaning

In [230]:
# Copy the dataframe
df_3 = df_2.copy()

#### 7.1- Handling with ID Columns

In [231]:
# List the date columns
date_columns = ['timestamp']  # Add any other date columns here

# Convert all date columns to retain only the year information
for col in date_columns:
    df_3[col + '_year'] = pd.to_datetime(df_3[col]).dt.year

# Optionally, drop the original date columns
df_3.drop(columns=date_columns, inplace=True)

#### 7.2- Drop Unnecessary Columns

In [232]:
# Drop all columns that contain 'ID' in their column name
df_4 = df_3.loc[:, ~df_3.columns.str.contains('ID')]

#### 7.3- Encoding

In [233]:
# Copy the dataset
df_5 = df_4.copy()

# Identify categorical columns
categorical_columns = df_5.select_dtypes(include=['object']).columns

# Apply Label Encoding to each categorical column
label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    df_5[col] = le.fit_transform(df_5[col])
    label_encoders[col] = le  # Store the label encoder for inverse transform later if needed

df_5.head(1)
df_5.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Index: 38133 entries, 0 to 7661
Data columns (total 284 columns):
 #    Column                                 Non-Null Count  Dtype  
---   ------                                 --------------  -----  
 0    full_sq                                38133 non-null  float64
 1    life_sq                                30574 non-null  float64
 2    floor                                  37966 non-null  float64
 3    max_floor                              28561 non-null  float64
 4    material                               28561 non-null  float64
 5    build_year                             23479 non-null  float64
 6    num_room                               28561 non-null  float64
 7    kitch_sq                               28561 non-null  float64
 8    state                                  23880 non-null  float64
 9    product_type                           38133 non-null  int32  
 10   sub_area                               38133 non-null  int32  

#### 7.4 Handling Missing Data

In [234]:
# Copy the dataset
df_6 = df_5.copy()

In [235]:
# Eksik değerleri kolonların ortalaması ile doldurun
df_6 = df_6.fillna(df_6.mean())

#### 7.5 Drop Outliers 1

In [236]:
# Copy the dataframe
df_7 = df_6.copy()

In [237]:
import scipy.stats as stats

def winsorize_outliers(df, limits):
    df_winsorized = df.copy()
    for column in df.select_dtypes(include=['float64', 'int64']).columns:
        df_winsorized[column] = stats.mstats.winsorize(df[column], limits=[limits, limits])
    return df_winsorized

# Outlier'ları sınırlandırın (alt ve üst %5'lik dilimi)
df_7 = winsorize_outliers(df_7, limits=0.2)

#### 7.6 Check Outliers Test 1

In [238]:
'''
# Sütunları küçük gruplar halinde bölmek için adım boyutunu belirleyin
step = 10  # Aynı anda kaç sütun gösterileceğini belirler
num_columns = len(df_7.columns)

for i in range(0, num_columns, step):
    plt.figure(figsize=(12, 6))  # Grafik boyutunu ayarlayın
    sns.boxplot(data=df_2.iloc[:, i:i+step])
    plt.xticks(rotation=45, fontsize=8)
    plt.show()
'''
print('')

#### 7.7 Drop Outliers 2

In [239]:

import pandas as pd
from scipy import stats

def remove_outliers_zscore(df, threshold):
    df_out = df.copy()  # Orijinal veri setini korumak için bir kopya alın
    for column in df_out.select_dtypes(include=['float64', 'int64']).columns:
        # Z-score'u hesapla ve doğrudan DataFrame üzerinde filtreleme yap
        df_out = df_out[(abs(stats.zscore(df_out[column].dropna())) < threshold) | df_out[column].isna()]
    return df_out

In [240]:
# Outlier'ları Z-score yöntemiyle temizleyin
df_8 = remove_outliers_zscore(df_6, threshold=5)

#### 7.8 Check Outliers Test 2

In [241]:
'''
# Sütunları küçük gruplar halinde bölmek için adım boyutunu belirleyin
step = 10  # Aynı anda kaç sütun gösterileceğini belirler
num_columns = len(df_8.columns)

for i in range(0, num_columns, step):
    plt.figure(figsize=(12, 6))  # Grafik boyutunu ayarlayın
    sns.boxplot(data=df_2.iloc[:, i:i+step])
    plt.xticks(rotation=45, fontsize=8)
    plt.show()
'''
print('')

#### 7.9 Drop Outliers 3

In [242]:
# Cleaning Function 
def boxplot_outliers_treatment(df):
    df_out = df.copy()
    
    for column in df_out.select_dtypes(include=['float64', 'int64']).columns:
        # IQR hesaplama
        Q1 = df_out[column].quantile(0.25)
        Q3 = df_out[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        # Aykırı değerleri sınırlandırma (Winsorization)
        df_out[column] = df_out[column].apply(lambda x: lower_bound if x < lower_bound else upper_bound if x > upper_bound else x)
        
        # Alternatif: Aykırı değerleri medyan ile doldurma
        # median_value = df_out[column].median()
        # df_out[column] = df_out[column].apply(lambda x: median_value if x < lower_bound or x > upper_bound else x)
        
    return df_out

In [243]:
# Outlier'ları Boxplot ve IQR kullanarak düzeltin
df_9 = boxplot_outliers_treatment(df_6)

In [244]:
df_9.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38133 entries, 0 to 7661
Columns: 284 entries, full_sq to timestamp_year
dtypes: float64(258), int32(16), int64(10)
memory usage: 80.6 MB


#### 7.10 Check Outliers Test 3

In [245]:
'''
# Sütunları küçük gruplar halinde bölmek için adım boyutunu belirleyin
step = 10  # Aynı anda kaç sütun gösterileceğini belirler
num_columns = len(df_9.columns)

for i in range(0, num_columns, step):
    plt.figure(figsize=(12, 6))  # Grafik boyutunu ayarlayın
    sns.boxplot(data=df_2.iloc[:, i:i+step])
    plt.xticks(rotation=45, fontsize=8)
    plt.show()
'''
print('')

#### 7.11 Drop Outliers 4

In [246]:
def mark_outliers_zscore(df, threshold=3):
    df_out = df.copy()  # Orijinal veri setini korumak için bir kopya alın
    for column in df.select_dtypes(include=['float64', 'int64']).columns:
        z_scores = stats.zscore(df[column].dropna())  # NaN değerleri hesaplamadan önce çıkar
        abs_z_scores = abs(z_scores)
        # Sadece outlier olan değerleri NaN olarak işaretleyin
        df_out.loc[abs_z_scores > threshold, column] = None
    return df_out

# Outlier'ları NaN olarak işaretleyin
df_outliers_marked = mark_outliers_zscore(df_6)

# Outlier'ları içeren satırları düşürmek için tüm satırdaki NaN değerlerini temizleyin
df_10 = df_outliers_marked.dropna()

In [247]:
df_10.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16692 entries, 1 to 7660
Columns: 284 entries, full_sq to timestamp_year
dtypes: float64(268), int32(16)
memory usage: 35.3 MB


#### 7.12 Check Outliers Test 4

In [248]:
'''
# Sütunları küçük gruplar halinde bölmek için adım boyutunu belirleyin
step = 10  # Aynı anda kaç sütun gösterileceğini belirler
num_columns = len(df_10.columns)

for i in range(0, num_columns, step):
    plt.figure(figsize=(12, 6))  # Grafik boyutunu ayarlayın
    sns.boxplot(data=df_2.iloc[:, i:i+step])
    plt.xticks(rotation=45, fontsize=8)
    plt.show()
'''
print('')

#### 7.13 Data Normalization

In [249]:
# Copy the dataframe
df_11 = df_10.copy()

In [250]:
from sklearn.preprocessing import MinMaxScaler

def normalize_data(df, target_column):
    scaler = MinMaxScaler()
    df_normalized = df.copy()
    # Hedef değişkeni hariç tutarak bağımsız değişkenleri normalleştir
    columns_to_normalize = df_normalized.select_dtypes(include=['float64', 'int64']).columns.difference([target_column])
    df_normalized[columns_to_normalize] = scaler.fit_transform(df_normalized[columns_to_normalize])
    return df_normalized

# Hedef değişkeni hariç tutarak verileri normalize edin
target_column = 'price_doc'  # Buraya hedef kolonun adını yazın
df_11 = normalize_data(df_11, target_column)


#### 7.14 Check Distributions 2

In [251]:
'''
# Check the visualization
df_sampled = df_11.sample(frac=0.2)  # Verinin %10'unu alıyoruz

num_plots = len(numeric_columns)
plots_per_page = 16  # 4x4 yerleşim
num_pages = math.ceil(num_plots / plots_per_page)

for page in range(num_pages):
    plt.figure(figsize=(15, 15))
    
    start_idx = page * plots_per_page
    end_idx = min(start_idx + plots_per_page, num_plots)
    
    for i, column in enumerate(numeric_columns[start_idx:end_idx], 1):
        plt.subplot(4, 4, i)
        sns.histplot(df_sampled[column], kde=False)
        plt.title(column)
    
    plt.tight_layout()
    plt.show()
'''
print('')


#### 7.15 Correlation Test

In [252]:
# Copy Dataset
df_12 = df_11.copy()

In [253]:
# Hedef değişkeni çıkartarak korelasyon matrisini hesapla
target_column = 'price_doc'  # Buraya hedef kolonun adını yazın
corr_matrix = df_12.drop(columns=[target_column]).corr().abs()

In [254]:
# Üst üçgenin maskelenmesi
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Yüksek korelasyonlu kolonları bulma ve çıkarma (örneğin, 0.8 ve üzeri)
to_drop = [column for column in upper.columns if any(upper[column] > 0.80)]

In [255]:
print(f"drop the most correlated {len(to_drop)} columns")

drop the most correlated 128 columns


In [256]:
# Gereksiz kolonları çıkarma
df_12 = df_12.drop(columns=to_drop)

In [257]:
df_12.head(3)

,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,product_type,sub_area,area_m,raion_popul,green_zone_part,indust_part,preschool_quota,school_education_centers_top_20_raion,hospital_beds_raion,healthcare_centers_raion,university_top_20_raion,sport_objects_raion,additional_education_raion,culture_objects_top_25,culture_objects_top_25_raion,shopping_centers_raion,office_raion,thermal_power_plant_raion,incineration_raion,oil_chemistry_raion,radiation_raion,railroad_terminal_raion,big_market_raion,nuclear_reactor_raion,detention_facility_raion,full_all,raion_build_count_with_material_info,build_count_block,build_count_wood,build_count_frame,build_count_brick,build_count_monolith,build_count_panel,build_count_foam,build_count_mix,build_count_1921-1945,build_count_1971-1995,build_count_after_1995,metro_min_avto,kindergarten_km,park_km,green_zone_km,industrial_km,water_treatment_km,cemetery_km,incineration_km,railroad_station_walk_km,public_transport_station_km,water_km,water_1line,mkad_km,ttk_km,big_road1_km,big_road1_1line,big_road2_km,railroad_1line,bus_terminal_avto_km,oil_chemistry_km,nuclear_reactor_km,power_transmission_line_km,ts_km,big_market_km,market_shop_km,fitness_km,swim_pool_km,ice_rink_km,detention_facility_km,shopping_centers_km,office_km,additional_education_km,big_church_km,church_synagogue_km,mosque_km,theater_km,catering_km,ecology,green_part_500,prom_part_500,office_count_500,office_sqm_500,trc_count_500,trc_sqm_500,cafe_count_500,cafe_sum_500_min_price_avg,cafe_count_500_na_price,cafe_count_500_price_500,cafe_count_500_price_1500,cafe_count_500_price_2500,cafe_count_500_price_4000,cafe_count_500_price_high,big_church_count_500,church_count_500,mosque_count_500,leisure_count_500,sport_count_500,market_count_500,green_part_1000,prom_part_1000,office_count_1000,office_sqm_1000,trc_count_1000,trc_sqm_1000,cafe_count_1000,cafe_sum_1000_min_price_avg,cafe_count_1000_na_price,cafe_count_1000_price_2500,cafe_count_1000_price_4000,cafe_count_1000_price_high,big_church_count_1000,church_count_1000,mosque_count_1000,leisure_count_1000,sport_count_1000,market_count_1000,trc_count_1500,trc_sqm_1500,cafe_sum_1500_min_price_avg,cafe_count_1500_na_price,cafe_count_1500_price_2500,cafe_count_1500_price_4000,cafe_count_1500_price_high,big_church_count_1500,church_count_1500,mosque_count_1500,leisure_count_1500,market_count_1500,trc_sqm_2000,cafe_sum_2000_min_price_avg,cafe_count_2000_price_4000,cafe_count_2000_price_high,church_count_2000,mosque_count_2000,leisure_count_2000,prom_part_3000,trc_sqm_3000,cafe_sum_3000_min_price_avg,cafe_count_3000_price_4000,cafe_count_3000_price_high,mosque_count_3000,leisure_count_3000,market_count_3000,prom_part_5000,cafe_sum_5000_min_price_avg,mosque_count_5000,market_count_5000,price_doc,timestamp_year
1,0.224422,0.125413,0.130435,0.41892,0.208597,0.547187,0.475211,0.090889,0.357217,0,70,0.115744,0.641209,0.537390,0.107777,0.399613,0.0,0.060856,0.2,0.0,0.333333,0.1,1,0.25,0.1875,0.000000,0,0,0,0,0,0,0,0,0.300646,0.331522,0.535484,0.004902,0.000000,0.211356,0.068966,0.288462,0.0,0.0,0.009091,0.407767,0.059524,0.051379,0.027974,0.044141,0.055534,0.329877,0.038904,0.793471,0.349541,0.216251,0.014360,0.356293,0,0.693861,0.110439,0.555971,0,0.437445,0,0.233007,0.248396,0.136027,0.119361,0.044044,0.213706,0.238748,0.116086,0.140713,0.492645,0.332047,0.100471,0.092275,0.207950,0.095426,0.164291,0.188140,0.229659,0.071817,0,0.417469,0.000000,0.0,0.0,0.0,0.0,0.185185,0.466667,0.0,0.1,0.000000,0.0,0.5,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.445596,0.001545,0.095238,0.178739,0.454545,0.187571,0.288889,0.309749,0.0,0.000000,0.333333,0.0,0.166667,0.222222,0.0,0.8,0.166667,0.0,0.500000,0.168973,0.414863,0.0,0.133333,0.25,0.0,0.083333,0.25,0.0,0.5,0.00,0.176775,0.408306,0.166667,0.0,0.20,0.0,0.4,0.779937,0.278155,0.303697,0.058824,0.0,0.0,0.315789,0.111111,0.990280,0.351038,0.5,0.70,6000000.0,2011
3,0.587459,0.330033,0.391304,0.41892,0.208597,0.547187,0.475211,0.090889,0.357217,0,65,0.1620

### 8- Building Model-0 // Benchmark

In [258]:
# Copy dataset
df_13 = df_12.copy()

In [259]:
df_13.info(1)

<class 'pandas.core.frame.DataFrame'>
Index: 16692 entries, 1 to 7660
Data columns (total 156 columns):
 #    Column                                 Dtype  
---   ------                                 -----  
 0    full_sq                                float64
 1    life_sq                                float64
 2    floor                                  float64
 3    max_floor                              float64
 4    material                               float64
 5    build_year                             float64
 6    num_room                               float64
 7    kitch_sq                               float64
 8    state                                  float64
 9    product_type                           int32  
 10   sub_area                               int32  
 11   area_m                                 float64
 12   raion_popul                            float64
 13   green_zone_part                        float64
 14   indust_part                            flo

#### 8.1- Logistic Regression

In [260]:
# Separate the target variable and independent variables
target_column = 'price_doc'  

X = df_13.drop(columns=[target_column])
y = df_13[target_column]


In [261]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [262]:
# Create the Linear Regression model
model = LinearRegression()

# Train the model on the training data
model.fit(X_train, y_train)

LinearRegression()

In [263]:
# Make predictions on the test data
y_pred = model.predict(X_test)

# Evaluate the model's performance
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")

Mean Squared Error: 4249879544852.8145
R^2 Score: 0.44098415551818104


#### 8.2- Ridge Regression

In [264]:
from sklearn.linear_model import Ridge

# Ridge Regression modelini oluşturma
ridge_model = Ridge(alpha=1.0)  # Alpha düzenleme parametresidir
ridge_model.fit(X_train, y_train)

# Tahmin yapma
y_pred_ridge = ridge_model.predict(X_test)

# Performansı değerlendirme
mse_ridge = mean_squared_error(y_test, y_pred_ridge)
r2_ridge = r2_score(y_test, y_pred_ridge)

print(f"Ridge Regression - Mean Squared Error: {mse_ridge}")
print(f"Ridge Regression - R^2 Score: {r2_ridge}")


Ridge Regression - Mean Squared Error: 4251866078411.1675
Ridge Regression - R^2 Score: 0.4407228531158437


#### 8.3- Lasso Regression

In [265]:
from sklearn.linear_model import Lasso

# Lasso Regression modelini oluşturma
lasso_model = Lasso(alpha=0.1)  # Alpha düzenleme parametresidir
lasso_model.fit(X_train, y_train)

# Tahmin yapma
y_pred_lasso = lasso_model.predict(X_test)

# Performansı değerlendirme
mse_lasso = mean_squared_error(y_test, y_pred_lasso)
r2_lasso = r2_score(y_test, y_pred_lasso)

print(f"Lasso Regression - Mean Squared Error: {mse_lasso}")
print(f"Lasso Regression - R^2 Score: {r2_lasso}")


Lasso Regression - Mean Squared Error: 4247317501678.2085
Lasso Regression - R^2 Score: 0.44132115865291355


c:\Users\ilyas\anaconda3\envs\ml_pred\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.672e+16, tolerance: 9.552e+12
  model = cd_fast.enet_coordinate_descent(


### 9- Feature Engineering 1

In [275]:
# Copy dataset
df_14 = df_13.copy()

In [276]:
# Clean the column names
df_14.columns = df_14.columns.str.replace(' ', '_').str.replace('-', '_').str.lower()

In [274]:
df_14.head(1)

,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,product_type,sub_area,area_m,raion_popul,green_zone_part,indust_part,preschool_quota,school_education_centers_top_20_raion,hospital_beds_raion,healthcare_centers_raion,university_top_20_raion,sport_objects_raion,additional_education_raion,culture_objects_top_25,culture_objects_top_25_raion,shopping_centers_raion,office_raion,thermal_power_plant_raion,incineration_raion,oil_chemistry_raion,radiation_raion,railroad_terminal_raion,big_market_raion,nuclear_reactor_raion,detention_facility_raion,full_all,raion_build_count_with_material_info,build_count_block,build_count_wood,build_count_frame,build_count_brick,build_count_monolith,build_count_panel,build_count_foam,build_count_mix,build_count_1921_1945,build_count_1971_1995,build_count_after_1995,metro_min_avto,kindergarten_km,park_km,green_zone_km,industrial_km,water_treatment_km,cemetery_km,incineration_km,railroad_station_walk_km,public_transport_station_km,water_km,water_1line,mkad_km,ttk_km,big_road1_km,big_road1_1line,big_road2_km,railroad_1line,bus_terminal_avto_km,oil_chemistry_km,nuclear_reactor_km,power_transmission_line_km,ts_km,big_market_km,market_shop_km,fitness_km,swim_pool_km,ice_rink_km,detention_facility_km,shopping_centers_km,office_km,additional_education_km,big_church_km,church_synagogue_km,mosque_km,theater_km,catering_km,ecology,green_part_500,prom_part_500,office_count_500,office_sqm_500,trc_count_500,trc_sqm_500,cafe_count_500,cafe_sum_500_min_price_avg,cafe_count_500_na_price,cafe_count_500_price_500,cafe_count_500_price_1500,cafe_count_500_price_2500,cafe_count_500_price_4000,cafe_count_500_price_high,big_church_count_500,church_count_500,mosque_count_500,leisure_count_500,sport_count_500,market_count_500,green_part_1000,prom_part_1000,office_count_1000,office_sqm_1000,trc_count_1000,trc_sqm_1000,cafe_count_1000,cafe_sum_1000_min_price_avg,cafe_count_1000_na_price,cafe_count_1000_price_2500,cafe_count_1000_price_4000,cafe_count_1000_price_high,big_church_count_1000,church_count_1000,mosque_count_1000,leisure_count_1000,sport_count_1000,market_count_1000,trc_count_1500,trc_sqm_1500,cafe_sum_1500_min_price_avg,cafe_count_1500_na_price,cafe_count_1500_price_2500,cafe_count_1500_price_4000,cafe_count_1500_price_high,big_church_count_1500,church_count_1500,mosque_count_1500,leisure_count_1500,market_count_1500,trc_sqm_2000,cafe_sum_2000_min_price_avg,cafe_count_2000_price_4000,cafe_count_2000_price_high,church_count_2000,mosque_count_2000,leisure_count_2000,prom_part_3000,trc_sqm_3000,cafe_sum_3000_min_price_avg,cafe_count_3000_price_4000,cafe_count_3000_price_high,mosque_count_3000,leisure_count_3000,market_count_3000,prom_part_5000,cafe_sum_5000_min_price_avg,mosque_count_5000,market_count_5000,price_doc,timestamp_year
1,0.224422,0.125413,0.130435,0.41892,0.208597,0.547187,0.475211,0.090889,0.357217,0,70,0.115744,0.641209,0.53739,0.107777,0.399613,0.0,0.060856,0.2,0.0,0.333333,0.1,1,0.25,0.1875,0.0,0,0,0,0,0,0,0,0,0.300646,0.331522,0.535484,0.004902,0.0,0.211356,0.068966,0.288462,0.0,0.0,0.009091,0.407767,0.059524,0.051379,0.027974,0.044141,0.055534,0.329877,0.038904,0.793471,0.349541,0.216251,0.01436,0.356293,0,0.693861,0.110439,0.555971,0,0.437445,0,0.233007,0.248396,0.136027,0.119361,0.044044,0.213706,0.238748,0.116086,0.140713,0.492645,0.332047,0.100471,0.092275,0.20795,0.095426,0.164291,0.18814,0.229659,0.071817,0,0.417469,0.0,0.0,0.0,0.0,0.0,0.185185,0.466667,0.0,0.1,0.0,0.0,0.5,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.445596,0.001545,0.095238,0.178739,0.454545,0.187571,0.288889,0.309749,0.0,0.0,0.333333,0.0,0.166667,0.222222,0.0,0.8,0.166667,0.0,0.5,0.168973,0.414863,0.0,0.133333,0.25,0.0,0.083333,0.25,0.0,0.5,0.0,0.176775,0.408306,0.166667,0.0,0.2,0.0,0.4,0.779937,0.278155,0.303697,0.058824,0.0,0.0,0.315789,0.111111,0.99028,0.351038,0.5,0.7,6000000.0,2011


#### 9.1 Combine the columns

In [279]:
# Combine cafe counts
cafe_count_columns = [col for col in df_14.columns if 'cafe_count_' in col and 'price' not in col]
df_14['total_cafe_count'] = df_14[cafe_count_columns].sum(axis=1)
df_14 = df_14.drop(columns=cafe_count_columns)

In [281]:
# Combine cafe prices
cafe_price_columns = [col for col in df_14.columns if 'cafe_sum_' in col or 'cafe_avg_price_' in col or ('cafe_count_' in col and 'price' in col)]
df_14['avg_cafe_price'] = df_14[cafe_price_columns].mean(axis=1)
df_14 = df_14.drop(columns=cafe_price_columns)

In [283]:
# Combine church counts
church_count_columns = [col for col in df_14.columns if 'church_count_' in col or 'big_church_count_' in col]
df_14['total_church_count'] = df_14[church_count_columns].sum(axis=1)
df_14 = df_14.drop(columns=church_count_columns)

In [284]:
# Combine mosque counts
mosque_count_columns = [col for col in df_14.columns if 'mosque_count_' in col]
df_14['total_mosque_count'] = df_14[mosque_count_columns].sum(axis=1)
df_14 = df_14.drop(columns=mosque_count_columns)

In [285]:
# Combine leisure counts
leisure_count_columns = [col for col in df_14.columns if 'leisure_count_' in col]
df_14['total_leisure_count'] = df_14[leisure_count_columns].sum(axis=1)
df_14 = df_14.drop(columns=leisure_count_columns)

In [286]:
# Combine market counts
market_count_columns = [col for col in df_14.columns if 'market_count_' in col]
df_14['total_market_count'] = df_14[market_count_columns].sum(axis=1)
df_14 = df_14.drop(columns=market_count_columns)

In [287]:
# Combine shopping center counts
shopping_center_columns = [col for col in df_14.columns if 'trc_count_' in col]
df_14['total_shopping_centers'] = df_14[shopping_center_columns].sum(axis=1)
df_14 = df_14.drop(columns=shopping_center_columns)

In [288]:
df_14.info(1)

<class 'pandas.core.frame.DataFrame'>
Index: 16692 entries, 1 to 7660
Data columns (total 111 columns):
 #    Column                                 Dtype  
---   ------                                 -----  
 0    full_sq                                float64
 1    life_sq                                float64
 2    floor                                  float64
 3    max_floor                              float64
 4    material                               float64
 5    build_year                             float64
 6    num_room                               float64
 7    kitch_sq                               float64
 8    state                                  float64
 9    product_type                           int32  
 10   sub_area                               int32  
 11   area_m                                 float64
 12   raion_popul                            float64
 13   green_zone_part                        float64
 14   indust_part                            flo

### 10- Building Model-1 vs. Benchmark

In [201]:
# Separate the target variable and independent variables
target_column = 'price_doc'  

X = df_14.drop(columns=[target_column])
y = df_14[target_column]


In [202]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [203]:
# Create the Linear Regression model
model = LinearRegression()

# Train the model on the training data
model.fit(X_train, y_train)

LinearRegression()

In [204]:
# Make predictions on the test data
y_pred = model.predict(X_test)

# Evaluate the model's performance
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")

Mean Squared Error: 4288706974657.939
R^2 Score: 0.43587691701118414


### 11- Feature Engineering 2

In [205]:
# Copy dataset
df_15 = df_14.copy()

In [207]:
df_15.info(1)

<class 'pandas.core.frame.DataFrame'>
Index: 16692 entries, 1 to 7660
Data columns (total 125 columns):
 #    Column                                 Dtype  
---   ------                                 -----  
 0    full_sq                                float64
 1    life_sq                                float64
 2    floor                                  float64
 3    max_floor                              float64
 4    material                               float64
 5    build_year                             float64
 6    num_room                               float64
 7    kitch_sq                               float64
 8    state                                  float64
 9    product_type                           int32  
 10   sub_area                               int32  
 11   area_m                                 float64
 12   raion_popul                            float64
 13   green_zone_part                        float64
 14   indust_part                            flo

#### 11.1 Combine the columns

In [ ]:
# Combining all cafe count averages into a single column
df_15['avg_cafe_count_total'] = df_15[['cafe_count_500', 'cafe_count_1000', 'cafe_count_1500', 'cafe_count_2000', 'cafe_count_3000', 'cafe_count_5000']].mean(axis=1)

In [ ]:
# Dropping the original columns after combining them
df_15 = df_15.drop(columns=['cafe_count_500', 'cafe_count_1000', 'cafe_count_1500', 
                            'cafe_count_2000', 'cafe_count_3000', 'cafe_count_5000'])

In [133]:
# Combining all church counts into a single column
df_15['total_church_count'] = df_15[['big_church_count_500', 'church_count_500', 'big_church_count_1000', 
                                     'church_count_1000', 'big_church_count_1500', 'church_count_1500', 
                                     'big_church_count_2000', 'church_count_2000', 'big_church_count_3000', 
                                     'church_count_3000', 'big_church_count_5000', 'church_count_5000']].sum(axis=1)

# Dropping the original columns after combining them
df_15 = df_15.drop(columns=['big_church_count_500', 'church_count_500', 'big_church_count_1000', 
                            'church_count_1000', 'big_church_count_1500', 'church_count_1500', 
                            'big_church_count_2000', 'church_count_2000', 'big_church_count_3000', 
                            'church_count_3000', 'big_church_count_5000', 'church_count_5000'])


KeyError: "['big_church_count_500', 'church_count_500', 'big_church_count_1000', 'church_count_1000', 'big_church_count_1500', 'church_count_1500', 'church_count_2000'] not in index"

In [ ]:
# Combining all mosque counts into a single column
df_15['total_mosque_count'] = df_15[['mosque_count_500', 'mosque_count_1000', 'mosque_count_1500', 
                                     'mosque_count_2000', 'mosque_count_3000', 'mosque_count_5000']].sum(axis=1)

# Dropping the original columns after combining them
df_15 = df_15.drop(columns=['mosque_count_500', 'mosque_count_1000', 'mosque_count_1500', 
                            'mosque_count_2000', 'mosque_count_3000', 'mosque_count_5000'])


In [ ]:
# Combining all leisure counts into a single column
df_15['total_leisure_count'] = df_15[['leisure_count_500', 'leisure_count_1000', 'leisure_count_1500', 
                                      'leisure_count_2000', 'leisure_count_3000', 'leisure_count_5000']].sum(axis=1)

# Dropping the original columns after combining them
df_15 = df_15.drop(columns=['leisure_count_500', 'leisure_count_1000', 'leisure_count_1500', 
                            'leisure_count_2000', 'leisure_count_3000', 'leisure_count_5000'])


In [ ]:
# Combining all market counts into a single column
df_15['total_market_count'] = df_15[['market_count_500', 'market_count_1000', 'market_count_1500', 
                                     'market_count_2000', 'market_count_3000', 'market_count_5000']].sum(axis=1)

# Dropping the original columns after combining them
df_15 = df_15.drop(columns=['market_count_500', 'market_count_1000', 'market_count_1500', 
                            'market_count_2000', 'market_count_3000', 'market_count_5000'])


In [ ]:
# Combining all shopping center counts into a single column
df_15['total_shopping_centers'] = df_15[['trc_count_500', 'trc_count_1000', 'trc_count_1500', 
                                         'trc_count_2000', 'trc_count_3000', 'trc_count_5000']].sum(axis=1)

# Dropping the original columns after combining them
df_15 = df_15.drop(columns=['trc_count_500', 'trc_count_1000', 'trc_count_1500', 
                            'trc_count_2000', 'trc_count_3000', 'trc_count_5000'])


### 8- Data Cleaning

#### 8.1- Drop Outliers

##### 8.1.1- Balance

##### 8.1.2- EstimatedSalary

#### 8.2- Standart Scaler

### 9- Building Alternative Models

#### 9.1- Model 1 - Decision Trees

#### 9.2- Model 2 - Random Forest

#### 9.3- Model 3 - Gradient Boosting Machines

#### 9.4- Model 4 - xgboost

#### 10- Handling Imbalanced Data

#### 11- Re-Train Models with Balanced Dataset 

### 12- Feature Engineering

#### 12.1 Correlation Test

#### 12.2 Recursive Feature Elimination

### 13- Hyperparameter Tuning

#### 13.1- Grid Search

#### 13.2- Random Search

### 14- Cross Validation